# Imports

In [1]:
import MySQLdb
from pymongo import MongoClient
import pandas
import numpy as np
import json
from pprint import pprint

# Conexão com a base no MongoDB

In [2]:
# Conexão com o MongoDB
client = MongoClient()
client = MongoClient('localhost', 27017)

# Cria uma base de dados no MongoDB
db_mongo = client['tf_f_dupla1_fim']

# Conexão com a base no MySQL

In [3]:
# Conexão com o MySQL
db = MySQLdb.connect("localhost","root","SENHA","tf_f_dupla1_fim")
cursor = db.cursor()

# Configurações do MySQL
db.set_character_set('utf8')
cursor.execute('SET NAMES utf8;')
cursor.execute('SET CHARACTER SET utf8;')
cursor.execute('SET character_set_connection=utf8;')

0

# Classe para ler e salvar os 'Movies' nas bases

In [4]:
# Cria a classe que armazena os filmes como json
class ReadMovie():
    def __init__(self, path):
        self.data = None
        self.json_data = None
        self.path = path
        self.insert = """INSERT INTO MOVIE VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        
        self.read_data()


    def read_data(self):
        self.data = pandas.read_csv(self.path,
                        usecols=['id', 'title', 'genres', 'overview', 'popularity', 'release_date', 'runtime'])
        self.data = self.data.replace(np.nan, None)
        self.data.drop_duplicates(subset ="id", keep = 'first', inplace = True)
        
        self.json_data = self.data.to_json(orient = "records")


    def save_data(self):
        aux = 0
        for idx, mydata in self.data.iterrows():
            idMovie = mydata['id']
            title = mydata['title']
            genres = json.dumps(mydata['genres'], ensure_ascii=False).encode('utf8')
            overview = str(mydata['overview'])
            popularity = mydata['popularity']
            runtime = mydata['runtime']
            release_date = mydata['release_date']
            
            ans = (idMovie, title, genres, overview, popularity, release_date, runtime)
            try:
                cursor.execute(self.insert, ans)
            except Exception as e:
                print(e)
        db.commit()


    def save_mongo(self):
        json_movies = json.loads(self.json_data)

        # Cria uma coleção
        movies = db_mongo['movie']
        movies.insert_many(json_movies)

# Função principal

In [5]:
def main():
    database_save = ReadMovie('./movies_metadata.csv')
    database_save.save_data()
    database_save.save_mongo()
    db.close()

In [6]:
main()

/home/regis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


(1265, "Data truncated for column 'idMovie' at row 1")
(1265, "Data truncated for column 'idMovie' at row 1")
(1265, "Data truncated for column 'idMovie' at row 1")


In [7]:
myquery = { "title": "Toy Story" }

mydoc = db_mongo['movie'].find(myquery)
for doc in mydoc:
    pprint(doc)

{'_id': ObjectId('5de40640bb3295ec6a49e10c'),
 'genres': "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, "
           "{'id': 10751, 'name': 'Family'}]",
 'id': '862',
 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's "
             'birthday brings Buzz Lightyear onto the scene. Afraid of losing '
             "his place in Andy's heart, Woody plots against Buzz. But when "
             'circumstances separate Buzz and Woody from their owner, the duo '
             'eventually learns to put aside their differences.',
 'popularity': 21.946943,
 'release_date': '1995-10-30',
 'runtime': 81.0,
 'title': 'Toy Story'}
